In [2]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()

# time series dataframe
ts_prices = pd.read_parquet("../data/transformed/validated_prices_2022-08-02-2024-08-02.parquet") # reads in parquet-file in specified path and returns its data in a dataframe
ts_prices

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,price_diff,volume_sma_5,volume_change,SMA_5,SMA_20,EMA_5,EMA_20,vol_5,daily_return
0,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-02 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
1,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-03 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
2,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-04 00:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
3,5.669698e+07,164.743200,163.210000,165.35,165.850000,163.00,491310.000000,2022-08-05 04:00:00,0.000000,5.694134e+07,0.000000,165.318000,169.103000,165.350000,165.350000,0.071554,0.000000
4,5.791877e+07,165.126767,164.263333,165.19,166.503333,163.40,507545.666667,2022-08-06 00:00:00,-0.160000,5.694134e+07,0.021549,165.318000,169.103000,165.296667,165.334762,0.071554,-0.000968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,3.515373e+07,218.131900,216.960000,218.24,219.300000,215.75,604680.000000,2024-07-29 04:00:00,0.093333,4.008290e+07,-0.042438,217.978000,225.414500,218.726415,221.994869,0.292077,0.000428
728,4.068162e+07,218.405900,219.190000,218.80,220.325000,216.12,584305.000000,2024-07-30 04:00:00,0.560000,3.812887e+07,0.157249,218.240000,224.705500,218.750943,221.690596,0.329983,0.002566
729,4.842297e+07,222.344100,221.440000,222.08,223.820000,220.63,668833.000000,2024-07-31 04:00:00,3.280000,3.984794e+07,0.190291,219.064000,224.431000,219.860629,221.727682,1.710839,0.014991
730,6.112524e+07,219.477300,224.370000,218.36,224.480000,217.02,876046.000000,2024-08-01 04:00:00,-3.720000,4.441905e+07,0.262319,219.125333,223.822000,219.360419,221.406950,1.670571,-0.016751


In [3]:
def get_cutoff_indicies(data, n_previous_days, step_size):
    stop_position = len(data)-1

    first_indx = 0
    last_indx = n_previous_days+1  # target index
    indicies = []

    while last_indx <= stop_position:
        indicies.append((first_indx, last_indx))

        first_indx += step_size
        last_indx += step_size
    
    return indicies

n_previous_days = 12  # make sure this constant across all uses
step_size = 1
indicies = get_cutoff_indicies(ts_prices, n_previous_days, step_size)
indicies[0:10]

[(0, 13),
 (1, 14),
 (2, 15),
 (3, 16),
 (4, 17),
 (5, 18),
 (6, 19),
 (7, 20),
 (8, 21),
 (9, 22)]

In [24]:
# (number_of_sequences, sequence_length, number_of_features)
# each sequence is an example, seq_len is number of days in each example, features is values per day
# number of cutoff indicies is each example
# for each cutoff indicies-pair get those days, create a matrix for each day add those days features as a row
# then add this matrix for this sequence to the total matrix as a single example
def transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size):
    indicies = get_cutoff_indicies(ts_prices, n_previous_days, step_size)
    indicies = indicies[:len(indicies)-1]  # exclude last pair
    num_seqs = len(indicies)
    print(f"Number of sequences/examples: {num_seqs}")

    features = []  
    targets = []

    for pair in indicies:
        start_indx = pair[0]  # start-index of day-row to end-index of day-row, all days in between this range is a sequence/example
        end_indx = pair[1]  # target indx of row

        sequence_df = ts_prices.iloc[start_indx: end_indx]  # get day-rows from start-indx to just before end-indx, each element is a day in current-sequence-day-example

        cur_seq_days_matrix = []  # each row is the values of features for each day in cur-sequence-example
        for _, day_row in sequence_df.iterrows():
            
            values = list(day_row.drop(labels="datetime").values)
            cur_seq_days_matrix.append(values)

        features.append(cur_seq_days_matrix)
        target_value = ts_prices.iloc[end_indx]["close_price"]   # target row is end-indx close-price
        targets.append(target_value)

        if pair == indicies[0]: # confirm last-val of seq is not same as the target next value
            print(f"Last val: {sequence_df.iloc[-1]["close_price"]}")
            print(f"Target val: {target_value}")

    features = np.array(features, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)
    return features, targets

print(indicies)
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)
print(f"\nConfirm number of examples: {len(features) == len(targets)}")
print(f"Num-features-in-each-day: {len(ts_prices.columns)-1}")
print(f"N-previous-days: {n_previous_days}")
print(f"Step-size: {step_size}")
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)
features


[(0, 13), (1, 14), (2, 15), (3, 16), (4, 17), (5, 18), (6, 19), (7, 20), (8, 21), (9, 22), (10, 23), (11, 24), (12, 25), (13, 26), (14, 27), (15, 28), (16, 29), (17, 30), (18, 31), (19, 32), (20, 33), (21, 34), (22, 35), (23, 36), (24, 37), (25, 38), (26, 39), (27, 40), (28, 41), (29, 42), (30, 43), (31, 44), (32, 45), (33, 46), (34, 47), (35, 48), (36, 49), (37, 50), (38, 51), (39, 52), (40, 53), (41, 54), (42, 55), (43, 56), (44, 57), (45, 58), (46, 59), (47, 60), (48, 61), (49, 62), (50, 63), (51, 64), (52, 65), (53, 66), (54, 67), (55, 68), (56, 69), (57, 70), (58, 71), (59, 72), (60, 73), (61, 74), (62, 75), (63, 76), (64, 77), (65, 78), (66, 79), (67, 80), (68, 81), (69, 82), (70, 83), (71, 84), (72, 85), (73, 86), (74, 87), (75, 88), (76, 89), (77, 90), (78, 91), (79, 92), (80, 93), (81, 94), (82, 95), (83, 96), (84, 97), (85, 98), (86, 99), (87, 100), (88, 101), (89, 102), (90, 103), (91, 104), (92, 105), (93, 106), (94, 107), (95, 108), (96, 109), (97, 110), (98, 111), (99, 11

array([[[ 5.6696984e+07,  1.6474319e+02,  1.6321001e+02, ...,
          1.6535001e+02,  7.1554177e-02,  0.0000000e+00],
        [ 5.6696984e+07,  1.6474319e+02,  1.6321001e+02, ...,
          1.6535001e+02,  7.1554177e-02,  0.0000000e+00],
        [ 5.6696984e+07,  1.6474319e+02,  1.6321001e+02, ...,
          1.6535001e+02,  7.1554177e-02,  0.0000000e+00],
        ...,
        [ 6.8039384e+07,  1.7107539e+02,  1.6982001e+02, ...,
          1.6647890e+02,  3.0757976e+00,  2.1425605e-02],
        [ 6.3390152e+07,  1.7159207e+02,  1.7038667e+02, ...,
          1.6704884e+02,  3.0654938e+00,  2.1111756e-03],
        [ 5.8740924e+07,  1.7210873e+02,  1.7095334e+02, ...,
          1.6759911e+02,  2.0051730e+00,  2.1067280e-03]],

       [[ 5.6696984e+07,  1.6474319e+02,  1.6321001e+02, ...,
          1.6535001e+02,  7.1554177e-02,  0.0000000e+00],
        [ 5.6696984e+07,  1.6474319e+02,  1.6321001e+02, ...,
          1.6535001e+02,  7.1554177e-02,  0.0000000e+00],
        [ 5.6696984e+07, 

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


X = features
Y = targets

model = Sequential()
model.add(Bidirectional(LSTM(units=10, return_sequences=True, input_shape=(X.shape[1], X.shape[2]))))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dense(1))  # Output layer for a single prediction

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X, Y, epochs=50, batch_size=32, validation_split=0.2)

predictions = model.predict(X)

# Example: print the first few predictions
print(targets[:5])
print(predictions[:5])


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 27380.8496 - val_loss: 33454.0781
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 24665.2969 - val_loss: 32442.2227
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 23719.2246 - val_loss: 31719.1211
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 23282.9473 - val_loss: 31036.6074
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 22562.7578 - val_loss: 30381.8047
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - loss: 21940.9414 - val_loss: 29747.9199
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - loss: 21327.9980 - val_loss: 29131.3535
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - loss: 20856.7695 - val_loss: 28528.5938
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 20626.6191 - val_loss: 27938.0547
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 19774.4258 - val_loss: 27362.1250
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 19329.066

In [41]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout

model = Sequential()
model.add(SimpleRNN(units=10, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))  # Dropout to prevent overfitting
model.add(SimpleRNN(units=30))  # Second RNN layer
model.add(Dense(1))  # Output layer for a single prediction

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X, Y, epochs=10, batch_size=32, validation_split=0.2)
predictions = model.predict(X)

# Example: print the first few predictions
print(targets[:5])
print(predictions[:5])

Epoch 1/10


/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 27711.4941 - val_loss: 36395.0547
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27394.4277 - val_loss: 35758.0156
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26830.9531 - val_loss: 35246.4805
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26480.9668 - val_loss: 34804.0859
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26023.2676 - val_loss: 34452.2070
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25668.1074 - val_loss: 34177.2383
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25640.0254 - val_loss: 33931.6680
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25316.8711 - val_loss: 33698.7031
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24797.6875 - val_loss: 33472.7188
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 24810.4629 - val_loss: 33251.2500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
[173.19 173.03 174.55 174.15 171.52]
[[12.447848]


In [40]:
X_flat = X.reshape(X.shape[0], -1)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the model
model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=(X_flat.shape[1],)))  # Input shape is (sequence_length * number_of_features,)
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(units=64, activation='relu'))


model.add(Dense(1))  # Output layer for a single prediction

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_flat, Y, epochs=10, batch_size=32, validation_split=0.2)

predictions = model.predict(X_flat)

print(targets[:5])
print(predictions[:5])

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 140112696442880.0000 - val_loss: 54672329015296.0000
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 40595766640640.0000 - val_loss: 9893673172992.0000
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19285615312896.0000 - val_loss: 3162523828224.0000
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11363340517376.0000 - val_loss: 2291676741632.0000
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8566870638592.0000 - val_loss: 1665333067776.0000
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6914778857472.0000 - val_loss: 1291770527744.0000
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5075327516672.0000 - val_loss: 1092190011392.0000
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4810976264192.0000 - val_loss: 1261350420480.0000
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3373449347072.0000 - val_loss: 1287489191936.0000
Epoch 10/10
18/18 ━━━━

In [38]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

num_samples, sequence_length, num_features = X.shape
X_train_flattened = X.reshape(num_samples, sequence_length * num_features)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_flattened, Y)

y_pred = model.predict(X_train_flattened)

mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")
# Print the first few predictions and actual values
print("Predictions:", y_pred[:5])
print("Actual values:", Y[:5])


Mean Squared Error: 0.7629116531530507
Predictions: [173.20205139 172.99509949 173.702052   175.07864731 172.23541946]
Actual values: [173.19 173.03 174.55 174.15 171.52]


In [16]:
# cannot combine because 3d:
# tabular_data = pd.DataFrame(features)
# tabular_data["target_close_price_next_day"] = pd.DataFrame(targets)
# tabular_data
# # each column is a dayx-featurey

# dont save because its 3d: save features/targets tabular data
# tabular_data.to_parquet("../data/transformed/tabular_prices_2022-08-02-2024-08-02.parquet")

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


Mean Squared Error: 0.9815958805094114
221


Random Forest MSE: 0.7365136091476493
